In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16149582631605422197]

In [13]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)
        
        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("drive/Colab Notebooks/Colab Testing/GAN_mnist_keras/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.510713, acc.: 62.50%] [G loss: 0.504791]
1 [D loss: 0.505300, acc.: 56.25%] [G loss: 0.590162]
2 [D loss: 0.400519, acc.: 71.88%] [G loss: 0.738487]
3 [D loss: 0.307044, acc.: 90.62%] [G loss: 0.937910]
4 [D loss: 0.273653, acc.: 93.75%] [G loss: 1.094664]
5 [D loss: 0.205260, acc.: 100.00%] [G loss: 1.198206]
6 [D loss: 0.200717, acc.: 96.88%] [G loss: 1.370893]
7 [D loss: 0.165185, acc.: 100.00%] [G loss: 1.469582]
8 [D loss: 0.164297, acc.: 100.00%] [G loss: 1.611933]
9 [D loss: 0.130891, acc.: 100.00%] [G loss: 1.739431]
10 [D loss: 0.133516, acc.: 96.88%] [G loss: 1.807240]
11 [D loss: 0.087717, acc.: 100.00%] [G loss: 1.918068]
12 [D loss: 0.111471, acc.: 100.00%] [G loss: 2.054948]
13 [D loss: 0.083486, acc.: 100.00%] [G loss: 2.002789]
14 [D loss: 0.107259, acc.: 100.00%] [G loss: 2.231953]
15 [D loss: 0.069183, acc.: 100.00%] [G loss: 2.305413]
16 [D loss: 0.071096, acc.: 100.00%] [G loss: 2.385226]
17 [D loss: 0.059032, acc.: 100.00%] [G loss: 2.444708]
18 [D los

100 [D loss: 0.009523, acc.: 100.00%] [G loss: 4.794017]
101 [D loss: 0.012360, acc.: 100.00%] [G loss: 4.612641]
102 [D loss: 0.011722, acc.: 100.00%] [G loss: 4.756517]
103 [D loss: 0.008457, acc.: 100.00%] [G loss: 4.510126]
104 [D loss: 0.008579, acc.: 100.00%] [G loss: 4.647977]
105 [D loss: 0.008501, acc.: 100.00%] [G loss: 4.433732]
106 [D loss: 0.016282, acc.: 100.00%] [G loss: 4.646357]
107 [D loss: 0.015566, acc.: 100.00%] [G loss: 4.641057]
108 [D loss: 0.012319, acc.: 100.00%] [G loss: 4.734161]
109 [D loss: 0.010955, acc.: 100.00%] [G loss: 4.811734]
110 [D loss: 0.026868, acc.: 100.00%] [G loss: 4.861585]
111 [D loss: 0.013477, acc.: 100.00%] [G loss: 5.055700]
112 [D loss: 0.021525, acc.: 100.00%] [G loss: 4.758181]
113 [D loss: 0.026256, acc.: 100.00%] [G loss: 4.845912]
114 [D loss: 0.008731, acc.: 100.00%] [G loss: 5.125791]
115 [D loss: 0.012773, acc.: 100.00%] [G loss: 4.936393]
116 [D loss: 0.008917, acc.: 100.00%] [G loss: 4.663512]
117 [D loss: 0.008932, acc.: 10

199 [D loss: 0.255904, acc.: 93.75%] [G loss: 2.746656]
200 [D loss: 0.241274, acc.: 90.62%] [G loss: 2.865190]
201 [D loss: 0.175832, acc.: 96.88%] [G loss: 3.196693]
202 [D loss: 0.630125, acc.: 68.75%] [G loss: 3.686684]
203 [D loss: 0.577441, acc.: 75.00%] [G loss: 3.134741]
204 [D loss: 0.291850, acc.: 93.75%] [G loss: 2.796853]
205 [D loss: 0.240711, acc.: 100.00%] [G loss: 2.158089]
206 [D loss: 0.372591, acc.: 78.12%] [G loss: 3.208405]
207 [D loss: 0.315517, acc.: 87.50%] [G loss: 2.508194]
208 [D loss: 0.351917, acc.: 87.50%] [G loss: 2.378848]
209 [D loss: 0.227579, acc.: 90.62%] [G loss: 3.818046]
210 [D loss: 0.608010, acc.: 65.62%] [G loss: 1.336309]
211 [D loss: 0.538353, acc.: 75.00%] [G loss: 3.315067]
212 [D loss: 0.541976, acc.: 65.62%] [G loss: 2.618055]
213 [D loss: 0.141863, acc.: 100.00%] [G loss: 2.942627]
214 [D loss: 0.201356, acc.: 93.75%] [G loss: 2.564276]
215 [D loss: 0.318391, acc.: 87.50%] [G loss: 2.866234]
216 [D loss: 0.322056, acc.: 84.38%] [G loss: 

...

29548 [D loss: 0.749845, acc.: 50.00%] [G loss: 0.860390]
29549 [D loss: 0.660611, acc.: 65.62%] [G loss: 0.851617]
29550 [D loss: 0.692396, acc.: 53.12%] [G loss: 0.802092]
29551 [D loss: 0.733641, acc.: 53.12%] [G loss: 0.885618]
29552 [D loss: 0.650364, acc.: 56.25%] [G loss: 0.836496]
29553 [D loss: 0.698679, acc.: 50.00%] [G loss: 0.813069]
29554 [D loss: 0.704416, acc.: 56.25%] [G loss: 0.911210]
29555 [D loss: 0.647907, acc.: 65.62%] [G loss: 0.835345]
29556 [D loss: 0.631840, acc.: 62.50%] [G loss: 0.944224]
29557 [D loss: 0.668125, acc.: 65.62%] [G loss: 0.811425]
29558 [D loss: 0.661906, acc.: 56.25%] [G loss: 0.889642]
29559 [D loss: 0.683810, acc.: 50.00%] [G loss: 0.837083]
29560 [D loss: 0.659400, acc.: 53.12%] [G loss: 0.785057]
29561 [D loss: 0.766663, acc.: 34.38%] [G loss: 0.740571]
29562 [D loss: 0.715285, acc.: 50.00%] [G loss: 0.825227]
29563 [D loss: 0.598467, acc.: 68.75%] [G loss: 0.841549]
29564 [D loss: 0.636495, acc.: 65.62%] [G loss: 0.855382]
29565 [D loss:

29661 [D loss: 0.665852, acc.: 68.75%] [G loss: 0.804919]
29662 [D loss: 0.636420, acc.: 59.38%] [G loss: 0.817160]
29663 [D loss: 0.671168, acc.: 59.38%] [G loss: 0.950547]
29664 [D loss: 0.651023, acc.: 59.38%] [G loss: 0.923721]
29665 [D loss: 0.745289, acc.: 43.75%] [G loss: 0.852070]
29666 [D loss: 0.636651, acc.: 75.00%] [G loss: 0.798240]
29667 [D loss: 0.663378, acc.: 65.62%] [G loss: 0.853932]
29668 [D loss: 0.654108, acc.: 68.75%] [G loss: 0.830549]
29669 [D loss: 0.718335, acc.: 53.12%] [G loss: 0.743094]
29670 [D loss: 0.686592, acc.: 62.50%] [G loss: 0.720253]
29671 [D loss: 0.621191, acc.: 68.75%] [G loss: 0.847238]
29672 [D loss: 0.671262, acc.: 56.25%] [G loss: 0.831036]
29673 [D loss: 0.692246, acc.: 53.12%] [G loss: 0.802062]
29674 [D loss: 0.630399, acc.: 71.88%] [G loss: 0.826240]
29675 [D loss: 0.625311, acc.: 68.75%] [G loss: 0.832478]
29676 [D loss: 0.715273, acc.: 59.38%] [G loss: 0.862417]
29677 [D loss: 0.633146, acc.: 68.75%] [G loss: 0.861418]
29678 [D loss:

29773 [D loss: 0.653195, acc.: 62.50%] [G loss: 0.867891]
29774 [D loss: 0.655107, acc.: 59.38%] [G loss: 0.872515]
29775 [D loss: 0.692236, acc.: 56.25%] [G loss: 0.828802]
29776 [D loss: 0.649903, acc.: 59.38%] [G loss: 0.751894]
29777 [D loss: 0.650606, acc.: 53.12%] [G loss: 0.795254]
29778 [D loss: 0.682905, acc.: 62.50%] [G loss: 0.894779]
29779 [D loss: 0.759326, acc.: 46.88%] [G loss: 0.836987]
29780 [D loss: 0.699970, acc.: 56.25%] [G loss: 0.853616]
29781 [D loss: 0.814514, acc.: 43.75%] [G loss: 0.782551]
29782 [D loss: 0.685274, acc.: 59.38%] [G loss: 0.787983]
29783 [D loss: 0.649018, acc.: 68.75%] [G loss: 0.860950]
29784 [D loss: 0.698306, acc.: 53.12%] [G loss: 0.886142]
29785 [D loss: 0.691013, acc.: 56.25%] [G loss: 0.836775]
29786 [D loss: 0.754910, acc.: 37.50%] [G loss: 0.904487]
29787 [D loss: 0.686214, acc.: 62.50%] [G loss: 0.921577]
29788 [D loss: 0.657246, acc.: 65.62%] [G loss: 0.883776]
29789 [D loss: 0.679900, acc.: 59.38%] [G loss: 0.900054]
29790 [D loss:

29884 [D loss: 0.654275, acc.: 59.38%] [G loss: 0.829486]
29885 [D loss: 0.652913, acc.: 56.25%] [G loss: 0.883043]
29886 [D loss: 0.623876, acc.: 65.62%] [G loss: 0.952580]
29887 [D loss: 0.579656, acc.: 75.00%] [G loss: 0.846028]
29888 [D loss: 0.736463, acc.: 46.88%] [G loss: 0.877745]
29889 [D loss: 0.660525, acc.: 56.25%] [G loss: 0.968510]
29890 [D loss: 0.652198, acc.: 65.62%] [G loss: 0.842679]
29891 [D loss: 0.725869, acc.: 46.88%] [G loss: 0.853421]
29892 [D loss: 0.648947, acc.: 62.50%] [G loss: 0.920747]
29893 [D loss: 0.615928, acc.: 65.62%] [G loss: 0.779120]
29894 [D loss: 0.631437, acc.: 68.75%] [G loss: 0.758406]
29895 [D loss: 0.634486, acc.: 68.75%] [G loss: 0.778702]
29896 [D loss: 0.672931, acc.: 56.25%] [G loss: 0.837695]
29897 [D loss: 0.685507, acc.: 56.25%] [G loss: 0.844983]
29898 [D loss: 0.704730, acc.: 56.25%] [G loss: 0.824917]
29899 [D loss: 0.648017, acc.: 59.38%] [G loss: 0.845664]
29900 [D loss: 0.652744, acc.: 75.00%] [G loss: 0.884198]
29901 [D loss:

29996 [D loss: 0.615779, acc.: 68.75%] [G loss: 0.948513]
29997 [D loss: 0.589278, acc.: 65.62%] [G loss: 0.940234]
29998 [D loss: 0.680750, acc.: 56.25%] [G loss: 0.871181]
29999 [D loss: 0.668771, acc.: 59.38%] [G loss: 0.807196]
